In [1]:
import xarray as xr
import json
from pandas import to_datetime, IntervalIndex
from pathlib import Path

from util import date_interval_endpoints as endpoints
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY 

home = str(Path.home())

In [2]:
cube = xr.open_zarr(home+"/stacks/13/2002/8920.zarr", chunks=None)["data"]

timestamps = [to_datetime(v['timestamp']) for v in json.loads(cube.json).values()]
cube.coords['slice_ids'] = timestamps
cube = cube.rename({'slice_ids': 'timestamps'})
cube = cube.sortby('timestamps')
cube = cube.assign_coords(bands=[b.split('_')[0] for b in cube.bands.data])
cube = xr.Dataset({b: cube.sel(dict(bands=b)).reset_coords('bands', drop=True) for b in cube.bands.data})
cube

<xarray.Dataset>
Dimensions:     (timestamps: 35, x: 256, y: 256)
Coordinates:
  * timestamps  (timestamps) datetime64[ns] 2019-06-02T09:58:14 ... 2019-08-29T10:07:49
Dimensions without coordinates: x, y
Data variables:
    B04         (timestamps, x, y) uint16 291 296 294 286 ... 4121 4117 4109
    B03         (timestamps, x, y) uint16 621 646 672 668 ... 4601 4599 4601
    B02         (timestamps, x, y) uint16 390 394 396 392 ... 5129 5137 5143
    B08         (timestamps, x, y) uint16 5122 5304 5430 5331 ... 5074 5093 5116

In [3]:
starttime = datetime(*min(timestamps).timetuple()[:3],0,0,0)
endtime = datetime(*max(timestamps).timetuple()[:3],0,0,0)

eps = endpoints(starttime, endtime, 16)
idx = IntervalIndex.from_arrays(eps[::2], eps[1::2])
idx

IntervalIndex([(2019-06-02, 2019-06-15 23:59:59], (2019-06-16, 2019-06-30 23:59:59], (2019-07-01, 2019-07-15 23:59:59], (2019-07-16, 2019-07-31 23:59:59], (2019-08-01, 2019-08-15 23:59:59], (2019-08-16, 2019-08-29 23:59:59]],
              closed='right',
              dtype='interval[datetime64[ns]]')

In [4]:
# Calculate values in time intervals
avg_cube = cube.groupby_bins('timestamps',bins=idx).mean('timestamps')

# Fake data
from numpy.random import random_sample
for b in ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12']:
    avg_cube[b] = avg_cube.B04
    avg_cube[b].data = random_sample(avg_cube.B04.shape)*4000

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) float64 1.812e+03 ... 3.099e+03
    B03              (timestamps_bins, x, y) float64 2.067e+03 ... 3.353e+03
    B02              (timestamps_bins, x, y) float64 2.029e+03 ... 3.623e+03
    B08              (timestamps_bins, x, y) float64 4.83e+03 ... 4.542e+03
    B05              (timestamps_bins, x, y) float64 6.135 ... 1.987e+03
    B06              (timestamps_bins, x, y) float64 3.364e+03 ... 485.8
    B07              (timestamps_bins, x, y) float64 470.2 ... 3.818e+03
    B8A              (timestamps_bins, x, y) float64 3.373e+03 ... 1.629e+03
    B11              (timestamps_bins, x, y) float64 2.946e+03 907.5 ... 399.1
    B12              (timestamps_bins, x, y) float64 659.0 3.245e+03 ... 388.0

In [5]:
def calculate_index(a,b):
    return (a-b)/(a+b)

ics = { # index components
  "NDVI":  ["B08", "B04"],
  "GNDVI": ["B08", "B03"],
  "BNDVI": ["B08", "B02"],
  "NDSI":  ["B11", "B12"],
  "NDWI":  ["B03", "B08"]
}

for ic in ics:
    avg_cube[ic] = calculate_index(avg_cube[ics[ic][0]], avg_cube[ics[ic][1]])
    
avg_cube['CVI'] = avg_cube.B08 * avg_cube.B05 / avg_cube.B03**2

avg_cube

<xarray.Dataset>
Dimensions:          (timestamps_bins: 6, x: 256, y: 256)
Coordinates:
  * timestamps_bins  (timestamps_bins) object (2019-06-02, 2019-06-15 23:59:59] ... (2019-08-16, 2019-08-29 23:59:59]
Dimensions without coordinates: x, y
Data variables:
    B04              (timestamps_bins, x, y) float64 1.812e+03 ... 3.099e+03
    B03              (timestamps_bins, x, y) float64 2.067e+03 ... 3.353e+03
    B02              (timestamps_bins, x, y) float64 2.029e+03 ... 3.623e+03
    B08              (timestamps_bins, x, y) float64 4.83e+03 ... 4.542e+03
    B05              (timestamps_bins, x, y) float64 6.135 ... 1.987e+03
    B06              (timestamps_bins, x, y) float64 3.364e+03 ... 485.8
    B07              (timestamps_bins, x, y) float64 470.2 ... 3.818e+03
    B8A              (timestamps_bins, x, y) float64 3.373e+03 ... 1.629e+03
    B11              (timestamps_bins, x, y) float64 2.946e+03 907.5 ... 399.1
    B12              (timestamps_bins, x, y) float64 659.0 3.245e+03 ... 388.0
    NDVI             (timestamps_bins, x, y) float64 0.4545 0.4568 ... 0.1888
    GNDVI            (timestamps_bins, x, y) float64 0.4005 0.4034 ... 0.1505
    BNDVI            (timestamps_bins, x, y) float64 0.4083 0.4138 ... 0.1125
    NDSI             (timestamps_bins, x, y) float64 0.6344 -0.563 ... 0.01407
    NDWI             (timestamps_bins, x, y) float64 -0.4005 -0.4034 ... -0.1505
    CVI              (timestamps_bins, x, y) float64 0.006935 1.262 ... 0.8026